<a href="https://colab.research.google.com/github/Manasa130306/CODSOFT/blob/main/spamdetection_task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
zip_path = r"C:\Users\killi\Downloads\archive(2).zip"


In [2]:
zip_path = "C:\\Users\\killi\\Downloads\\archive(2).zip"


In [3]:
from google.colab import files
uploaded = files.upload()


Saving archive (2).zip to archive (2).zip


In [22]:
import zipfile
import os
zip_path = "archive (2).zip"
extract_path = "sms_dataset"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed!")
print("Extracted files:", os.listdir(extract_path))


Extraction completed!
Extracted files: ['spam.csv']


In [6]:
import os
print(os.listdir())


['.config', 'archive (2).zip', 'sample_data']


In [8]:
import pandas as pd
data = pd.read_csv("sms_dataset/spam.csv", encoding="latin-1")
print(data.head())
print("\nColumns:", data.columns)


     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  

Columns: Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')


In [9]:

df = data[['v1', 'v2']].rename(columns={'v1': 'label', 'v2': 'message'})
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

print(df.head())
print("\nDataset size:", df.shape)
print("Class distribution:\n", df['label'].value_counts())


   label                                            message
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...

Dataset size: (5572, 2)
Class distribution:
 label
0    4825
1     747
Name: count, dtype: int64


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
X_train, X_test, y_train, y_test = train_test_split(
    df['message'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.968609865470852

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       966
           1       1.00      0.77      0.87       149

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.97      1115



In [20]:
import joblib
joblib.dump(model, "spam_classifier_nb.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

print(" Model and vectorizer saved!")


 Model and vectorizer saved!


In [19]:
def predict_message(message):
    msg_tfidf = vectorizer.transform([message])
    prediction = model.predict(msg_tfidf)[0]
    return "Spam " if prediction == 1 else "Ham "
print(predict_message("Congratulations! You won a free lottery ticket."))
print(predict_message("Hey, are we still meeting for lunch today?"))


Ham 
Ham 


In [14]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.9)


In [15]:
# Predict on test set
y_pred = model.predict(X_test_tfidf)

# Compare actual vs predicted for first 10 messages
for i in range(10):
    print("Message:", X_test.iloc[i])
    print("Actual label:", "Spam" if y_test.iloc[i] == 1 else "Ham")
    print("Predicted label:", "Spam" if y_pred[i] == 1 else "Ham")
    print("-"*50)


Message: Oh right, ok. I'll make sure that i do loads of work during the day!  got a really nasty cough today and is dry n shot so that should really help it!
Actual label: Ham
Predicted label: Ham
--------------------------------------------------
Message: I am in tirupur.  call you da.
Actual label: Ham
Predicted label: Ham
--------------------------------------------------
Message: No that just means you have a fat head
Actual label: Ham
Predicted label: Ham
--------------------------------------------------
Message: You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327
Actual label: Spam
Predicted label: Spam
--------------------------------------------------
Message: Come aftr  &lt;DECIMAL&gt; ..now i m cleaning the house
Actual label: Ham
Predicted label: Ham
--------------------------------------------------
Message: Friendship poem: Dear O Dear U R Not Near But I Can Hear Dont Get Fear Live With Cheer No More Tear U R Always my Dear. Gud ni8
Actual label: Ham
Pr